Demand profiles:
Scenario 1:

2030: 25% population switch to e-cooking

2040: 50% population switch to e-cooking

Scenario 2:

2030: 50% switch
xxxxx
2040: 100% switch

Adds load during typical cooking times (morning, lunch, evening) with assumed per capita cooking demand pattern.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Load baseline demand profile
df = pd.read_csv("demand_profiles - Copy.csv", parse_dates=['time'])
df.set_index('time', inplace=True)

# Load regional population data (2020 values)
pop_data = pd.read_csv("C:/Users/hie/pe_tan/pypsa-earth/Script_analysis/matched_population_by_region.csv")
pop_data = pop_data.set_index("Region")
pop_2020_total = pop_data["Population"].sum()
pop_data["Weight"] = pop_data["Population"] / pop_2020_total

# Parameters
population_2020=60051409 # 'Total Population, as of 1 January 2020 in millions UN prediction From: https://population.un.org/wpp/downloads?folder=Standard%20Projections&group=Most%20used
population_2030 = 79837087  # 'Total Population, as of 1 January 2030 in millions UN prediction From: https://population.un.org/wpp/downloads?folder=Standard%20Projections&group=Most%20used
population_2040 = 102778984 # 'Total Population, as of 1 January 2040 in millions UN prediction From: https://population.un.org/wpp/downloads?folder=Standard%20Projections&group=Most%20used
persons_per_household = 4.6   # average household size in Tanzania from HBS https://www.nbs.go.tz/nbs/takwimu/hbs/2017_18_HBS_Key_Indicators_Report_Engl.pdf
households_2030 = population_2030 / persons_per_household
households_2040 = population_2040 / persons_per_household

#




# Scale 2020 regional populations to 2030 and 2040
pop_data["Pop_2030"] = pop_data["Weight"] * population_2030
pop_data["Pop_2040"] = pop_data["Weight"] * population_2040

#Table 1 data (total peak demand and share of eCook in it) from https://www.researchgate.net/publication/352703752_An_Overview_of_the_Technical_Challenges_Facing_the_Deployment_of_Electric_Cooking_on_Hybrid_PVDiesel_Mini-Grid_in_Rural_Tanzania-A_Case_Study_Simulation
peak_data = {
    0.2: {'total': 16.07, 'share': 0.9001},
    0.5: {'total': 33.89, 'share': 0.9530},
    0.8: {'total': 55.49, 'share': 0.9713},
    1.0: {'total': 67.29, 'share': 0.9764},}



# Calculate eCook-only peak per household
ecook_peak_by_adoption = {
    round(k, 2): (v['total'] * v['share']) / 108 for k, v in peak_data.items()
}

# Fixed diurnal cooking profile
cooking_profile_fixed = {
    'morning': {'hours': [7, 8], 'fraction': 0.25},
    'lunch': {'hours': [12, 13], 'fraction': 0.30},
    'evening': {'hours': [18, 19, 20], 'fraction': 0.45}
}

def generate_ecooking_profile_population(df, year, adoption_rate):
    if year == 2030:
        pop_col = "Pop_2030"
    elif year == 2040:
        pop_col = "Pop_2040"
    else:
        raise ValueError("Unsupported year: use 2030 or 2040")

    hh_using_ecook = (pop_data[pop_col] / persons_per_household) * adoption_rate
    peak_per_hh_kw = ecook_peak_by_adoption.get(round(adoption_rate, 2))
    if peak_per_hh_kw is None:
        raise KeyError(f"Adoption rate {adoption_rate} not defined in ecook_peak_by_adoption")

    regional_peaks_kw = hh_using_ecook * peak_per_hh_kw

    added_load = pd.Series(0, index=df.index)
    for period, details in cooking_profile_fixed.items():
        for hour in details['hours']:
            added_load[df.index.hour == hour] += (
                details['fraction'] / len(details['hours'])
            )

    # Distribute demand across regions
    added_per_region = pd.DataFrame(index=df.index)
    for region in df.columns:
        if region in regional_peaks_kw:
            added_per_region[region] = added_load * regional_peaks_kw[region]
        else:
            added_per_region[region] = 0.0

    return df + added_per_region

# Define scenarios
scenarios = {
    's1_2030': (2030, 0.20),
    's1_2040': (2040, 0.50),
    's2_2030': (2030, 0.50),
    's2_2040': (2040, 1.00)
}

# Generate and save profiles
for name, (year, adoption) in scenarios.items():
    adjusted_df = generate_ecooking_profile_population(df, year, adoption)
    adjusted_df.to_csv(f"adjusted_demand_{name}_regional.csv")


C:\Users\hie\AppData\Local\Temp\ipykernel_17144\2608354106.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125
 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125
 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125
 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125
 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125
 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125
 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125
 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125
 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125
 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125
 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125
 0

Transmission Impact Analysis Script 

In [ ]:

def analyze_transmission_impact(network, label):
    network.lopf(network.snapshots)
    line_loading = network.lines_t.p0.abs().div(network.lines.s_nom, axis=0)
    max_loading = line_loading.max().max()
    lost_load = network.loads_t.p.sum().clip(upper=0).sum()

    print(f"Scenario {label}: Max Line Loading = {max_loading:.2f}, Lost Load = {lost_load:.2f} MW")

# Visualization for all scenarios
def plot_scenario_comparison(base_df, scenario_dfs, scenario_labels):
    plt.figure(figsize=(10, 5))
    daily_base = base_df.resample('D').max().sum(axis=1)
    plt.plot(daily_base.index, daily_base, label='Baseline', linestyle='--')
    for df, label in zip(scenario_dfs, scenario_labels):
        daily_peak = df.resample('D').max().sum(axis=1)
        plt.plot(daily_peak.index, daily_peak, label=label)

    plt.title('Daily Peak Load Comparison Across Scenarios')
    plt.ylabel('Total Daily Peak Load (MW)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Run plot
plot_scenario_comparison(df,
                         [df_s1_2030, df_s1_2040, df_s2_2030, df_s2_2040],
                         ['S1-2030 (25%)', 'S1-2040 (50%)', 'S2-2030 (50%)', 'S2-2040 (100%)'])